This program assigns roles to team members for the project. 

Conditions:
1. Team will rotate team roles in "cycles".
2. The length of each cycle may be two weeks or more, as determined by the team.
3. Each role has an expert who will guide the team members assigned to the role. 
4. Each team member may be assigned to more than one role at any given time. 
5. No team members, besides the experts, will be assigned to the same role in two consecutive cycles. 
6. Experts may be assigned to other roles just like any other team member. 

In [2]:
import pandas as pd
import numpy as np
import random
random.seed(42)

# Assign Roles 

This part of the code is used to assign roles to the team members from scratch

Instantiate list of members

In [3]:
members = [
            "Oluwaseyi Adaramola",
            "Farook Ajose",
            "Malik Akintola",
            "Ibukun Akinleye",
            "Samson Ilesanmi",
            "Oluwaseunfunmi Alarima",
            "Damilola Omokehinde",
            "Samuel Olayiwola",
            "Adejuwon Adegite",
            "Kafilat Akinyemi",
            "Chibike Enibe",
            "Ayodeji Abatan",
            "Divine Nkwocha",
            "AbdulQahhar Okenla",
        ]

members

['Oluwaseyi Adaramola',
 'Farook Ajose',
 'Malik Akintola',
 'Ibukun Akinleye',
 'Samson Ilesanmi',
 'Oluwaseunfunmi Alarima',
 'Damilola Omokehinde',
 'Samuel Olayiwola',
 'Adejuwon Adegite',
 'Kafilat Akinyemi',
 'Chibike Enibe',
 'Ayodeji Abatan',
 'Divine Nkwocha',
 'AbdulQahhar Okenla']

Instantiate available roles

In [4]:
roles = pd.DataFrame({
    "role": ["Project Manager",
            "Data Analyst (Python)",
            "Data Scientist/Statistician",
            "Visualization Specialist (Power BI)",
            "Technical Writer",
            "Publicity Coordinator",],
    "num_people": [2, 4, 4, 4, 4, 3],
    "expert": ["Farook Ajose",
               "Malik Akintola",
               "Ayodeji Abatan",
               "Samson Ilesanmi",
               "Farook Ajose",
               "AbdulQahhar Okenla"]
    })
roles

,role,num_people,expert
0,Project Manager,2,Farook Ajose
1,Data Analyst (Python),4,Malik Akintola
2,Data Scientist/Statistician,4,Ayodeji Abatan
3,Visualization Specialist (Power BI),4,Samson Ilesanmi
4,Technical Writer,4,Farook Ajose
5,Publicity Coordinator,3,AbdulQahhar Okenla


Write function to assign roles

In [5]:
num_cycles = 6

def assign_roles(row, cycles=num_cycles, team=members):
    """Assign team members to different roles for each rotation cycle, ensuring that an expert in the area is present to guide them. 

    Args:
        row (pandas.Series): A row of the `roles` dataframe containing the role details.
        cycles (int, optional): Number of rotation cycles to assign the roles for. Defaults to the value of `num_cycles`.
        team (list, optional): A list of team members to select from. Defaults to the value of `members`.

    Returns:
        expert_cycle_dict (dict): A dictionary containing the assigned members for each cycle, with keys in the format "Cycle x" where x is the cycle number.
    """
    
    # Remove expert from sample list 
    expert = row['expert']
    no_expert_team = team.copy()
    no_expert_team.remove(expert)

    # Create dictionary of assigned members for each cycle
    cycle_dict = {
        f"Cycle {cycle}":  np.random.choice(no_expert_team, size=row["num_people"]-1, replace=False)
        for cycle in range(1, cycles+1)
    }
    
    # Remove common values between consecutive cycles
    for i in range(1, cycles):
        current_cycle = cycle_dict[f"Cycle {i}"]
        next_cycle = cycle_dict[f"Cycle {i+1}"]
        common_members = np.intersect1d(current_cycle, next_cycle)
        for member in common_members:
            new_member = np.random.choice(no_expert_team, size=1)[0]
            while new_member in next_cycle:
                new_member = np.random.choice(no_expert_team, size=1)[0]
            next_cycle = np.where(next_cycle == member, new_member, next_cycle)
        cycle_dict[f"Cycle {i+1}"] = next_cycle
        
    # Append expert to list and convert to string    
    expert_cycle_dict = {
        f"Cycle {cycle}":  ", ".join(np.append(expert, cycle_dict[f"Cycle {cycle}"]))
        for cycle in range(1, cycles+1)
    }

    return expert_cycle_dict
        
# Test function to see outputs.     
assign_roles(roles.iloc[2, :])

{'Cycle 1': 'Ayodeji Abatan, Oluwaseunfunmi Alarima, Samuel Olayiwola, Chibike Enibe',
 'Cycle 2': 'Ayodeji Abatan, Damilola Omokehinde, Oluwaseyi Adaramola, Kafilat Akinyemi',
 'Cycle 3': 'Ayodeji Abatan, Samuel Olayiwola, Chibike Enibe, Damilola Omokehinde',
 'Cycle 4': 'Ayodeji Abatan, Divine Nkwocha, Ibukun Akinleye, Malik Akintola',
 'Cycle 5': 'Ayodeji Abatan, Adejuwon Adegite, Samuel Olayiwola, Oluwaseunfunmi Alarima',
 'Cycle 6': 'Ayodeji Abatan, Divine Nkwocha, Chibike Enibe, Ibukun Akinleye'}

Update dataframe with function

In [8]:
roles[[f"Cycle {cycle}" for cycle in range(1, num_cycles+1)]] = roles.apply(func=assign_roles, axis=1, result_type='expand')
roles

,role,num_people,expert,Cycle 1,Cycle 2,Cycle 3,Cycle 4,Cycle 5,Cycle 6
0,Project Manager,2,Farook Ajose,"Farook Ajose, Kafilat Akinyemi","Farook Ajose, Ayodeji Abatan","Farook Ajose, Oluwaseyi Adaramola","Farook Ajose, Samson Ilesanmi","Farook Ajose, Adejuwon Adegite","Farook Ajose, Malik Akintola"
1,Data Analyst (Python),4,Malik Akintola,"Malik Akintola, Samson Ilesanmi, Kafilat Akiny...","Malik Akintola, Divine Nkwocha, Adejuwon Adegi...","Malik Akintola, Farook Ajose, Ayodeji Abatan, ...","Malik Akintola, AbdulQahhar Okenla, Oluwaseunf...","Malik Akintola, Ibukun Akinleye, Samuel Olayiw...","Malik Akintola, AbdulQahhar Okenla, Ayodeji Ab..."
2,Data Scientist/Statistician,4,Ayodeji Abatan,"Ayodeji Abatan, Oluwaseunfunmi Alarima, Oluwas...","Ayodeji Abatan, Chibike Enibe, Adejuwon Adegit...","Ayodeji Abatan, Farook Ajose, AbdulQahhar Oken...","Ayodeji Abatan, Chibike Enibe, Adejuwon Adegit...","Ayodeji Abatan, AbdulQahhar Okenla, Samuel Ola...","Ayodeji Abatan, Adejuwon Adegite, Oluwaseyi Ad..."
3,Visualization Specialist (Power BI),4,Samson Ilesanmi,"Samson Ilesanmi, Chibike Enibe, Kafilat Akinye...","Samson Ilesanmi, AbdulQahhar Okenla, Oluwaseun...","Samson Ilesanmi, Adejuwon Adegite, Samuel Olay...","Samson Ilesanmi, Malik Akintola, Oluwaseyi Ada...","Samson Ilesanmi, Divine Nkwocha, Ibukun Akinle...","Samson Ilesanmi, Oluwaseyi Adaramola, Ayodeji ..."
4,Technical Writer,4,Farook Ajose,"Farook Ajose, AbdulQahhar Okenla, Oluwaseunfun...","Farook Ajose, Adejuwon Adegite, Oluwaseyi Adar...","Farook Ajose, Ibukun Akinleye, Damilola Omokeh...","Farook Ajose, Kafilat Akinyemi, Adejuwon Adegi...","Farook Ajose, Divine Nkwocha, Oluwaseyi Adaram...","Farook Ajose, Adejuwon Adegite, Samuel Olayiwo..."
5,Publicity Coordinator,3,AbdulQahhar Okenla,"AbdulQahhar Okenla, Malik Akintola, Ayodeji Ab...","AbdulQahhar Okenla, Farook Ajose, Samson Ilesanmi","AbdulQahhar Okenla, Kafilat Akinyemi, Chibike ...","AbdulQahhar Okenla, Adejuwon Adegite, Malik Ak...","AbdulQahhar Okenla, Ayodeji Abatan, Kafilat Ak...","AbdulQahhar Okenla, Oluwaseyi Adaramola, Adeju..."


Summarize the roles for each member per cycle

In [51]:
def summarize_roles(df, members=members, cycles=num_cycles):
    role_summary = {}
    for member in members:
        member_roles = []
        for cycle in range(1, cycles+1):
            assigned_roles = ", ".join(df["role"][df[f"Cycle {cycle}"].str.contains(member)])
            member_roles.append(assigned_roles)
        role_summary[member] = member_roles
        
    df = pd.DataFrame(role_summary).T
    df.columns = [f"Cycle {cycle}" for cycle in range(1, cycles+1)]
    
    return df

Save dataframe to CSV

In [7]:
# roles.to_csv("assigned_roles.csv", index=False)

# Update Existing Roles 

This part of the code is used to update the existing roles in a CSV.

In [50]:
current_roles = pd.read_csv("assigned_roles.csv")
current_roles.head()

,role,num_people,expert,Cycle 1,Cycle 2,Cycle 3,Cycle 4,Cycle 5,Cycle 6
0,Data Analyst (Python),4,Malik Akintola,"Malik Akintola, Chibike Enibe, AbdulQahhar Oke...","Malik Akintola, Samson Ilesanmi, Kafilat Akiny...","Malik Akintola, Adejuwon Adegite, Chibike Enib...","Malik Akintola, AbdulQahhar Okenla, Farook Ajo...","Malik Akintola, Adejuwon Adegite, Ayodeji Abat...","Malik Akintola, Farook Ajose, Chibike Enibe, D..."
1,Data Scientist/Statistician,4,Ayodeji Abatan,"Ayodeji Abatan, AbdulQahhar Okenla, Divine Nkw...","Ayodeji Abatan, Kafilat Akinyemi, Chibike Enib...","Ayodeji Abatan, Ibukun Akinleye, Oluwaseyi Ada...","Ayodeji Abatan, Oluwaseunfunmi Alarima, Samson...","Ayodeji Abatan, Samuel Olayiwola, Farook Ajose...","Ayodeji Abatan, Malik Akintola, Oluwaseyi Adar..."
2,Visualization Specialist (Power BI),4,Samson Ilesanmi,"Samson Ilesanmi, Ayodeji Abatan, Damilola Omok...","Samson Ilesanmi, Ibukun Akinleye, Farook Ajose...","Samson Ilesanmi, Samuel Olayiwola, Divine Nkwo...","Samson Ilesanmi, Ayodeji Abatan, AbdulQahhar O...","Samson Ilesanmi, Ibukun Akinleye, Oluwaseunfun...","Samson Ilesanmi, AbdulQahhar Okenla, Divine Nk..."
3,Technical Writer,4,Farook Ajose,"Farook Ajose, Oluwaseunfunmi Alarima, AbdulQah...","Farook Ajose, Samson Ilesanmi, Adejuwon Adegit...","Farook Ajose, Chibike Enibe, Samuel Olayiwola,...","Farook Ajose, Oluwaseyi Adaramola, AbdulQahhar...","Farook Ajose, Ibukun Akinleye, Chibike Enibe, ...","Farook Ajose, AbdulQahhar Okenla, Oluwaseyi Ad..."
4,Publicity Coordinator,3,AbdulQahhar Okenla,"AbdulQahhar Okenla, Ibukun Akinleye, Samuel Ol...","AbdulQahhar Okenla, Damilola Omokehinde, Adeju...","AbdulQahhar Okenla, Malik Akintola, Ibukun Aki...","AbdulQahhar Okenla, Samson Ilesanmi, Adejuwon ...","AbdulQahhar Okenla, Ibukun Akinleye, Kafilat A...","AbdulQahhar Okenla, Oluwaseyi Adaramola, Divin..."


In [53]:
current_role_summary = summarize_roles(current_roles)
current_role_summary

,Cycle 1,Cycle 2,Cycle 3,Cycle 4,Cycle 5,Cycle 6
Oluwaseyi Adaramola,Data Analyst (Python),,"Data Scientist/Statistician, Visualization Spe...",Technical Writer,,"Data Scientist/Statistician, Technical Writer,..."
Farook Ajose,Technical Writer,"Data Scientist/Statistician, Visualization Spe...",Technical Writer,"Data Analyst (Python), Technical Writer","Data Scientist/Statistician, Technical Writer","Data Analyst (Python), Technical Writer"
Malik Akintola,"Data Analyst (Python), Data Scientist/Statisti...",Data Analyst (Python),"Data Analyst (Python), Publicity Coordinator","Data Analyst (Python), Visualization Specialis...",Data Analyst (Python),"Data Analyst (Python), Data Scientist/Statisti..."
Ibukun Akinleye,Publicity Coordinator,"Data Analyst (Python), Visualization Specialis...","Data Scientist/Statistician, Technical Writer,...",,"Visualization Specialist (Power BI), Technical...",
Samson Ilesanmi,Visualization Specialist (Power BI),"Data Analyst (Python), Visualization Specialis...",Visualization Specialist (Power BI),"Data Scientist/Statistician, Visualization Spe...","Visualization Specialist (Power BI), Technical...",Visualization Specialist (Power BI)
Oluwaseunfunmi Alarima,Technical Writer,,,Data Scientist/Statistician,Visualization Specialist (Power BI),
Damilola Omokehinde,Visualization Specialist (Power BI),Publicity Coordinator,Data Scientist/Statistician,Data Analyst (Python),Visualization Specialist (Power BI),Data Analyst (Python)
Samuel Olayiwola,Publicity Coordinator,,"Data Analyst (Python), Visualization Specialis...",,Data Scientist/Statistician,
Adejuwon Adegite,,"Technical Writer, Publicity Coordinator",Data Analyst (Python),Publicity Coordinator,Data Analyst (Python),
Kafilat Akinyemi,Visualization Specialist (Power BI),"Data Analyst (Python), Data Scientist/Statisti...",,,"Data Analyst (Python), Publicity Coordinator",Data Scientist/Statistician
